# Chatmodel




## Aim:
1. Create chat model that can work with both HuggingFace Pipeline and BaseChatModel. Support tool calling.


## Known:
1. ChatHuggingFace and TextGenerationPipeline do NOT have tools option when call apply_chat_template.
2. Chat model automatically convert list[dict] message into langchain Message type.

## Directions:
1. Should we use HuggingFacePipeLine or TextGenerationPipeline or just Model and Tokenizer as a backend ? Just use Model and Tokenizer
for more abstraction.
2. TextGenerationPipeline offer an easy accessing to those and has some addition process for pre and post process.
We need change the behavior of this pipeline

2. FLow: apply_chat_template first, then pass it to TextGenerationPipeline as a STRING (Pipeline not apply_chat_template anymore).

We need to parse the tool output format from LLM ( xml and json) to form used by apply_chat_template or langchain

In [1]:
from drafts.core import create_test_textgen_pipeline
hf_pipeline = create_test_textgen_pipeline()

Device set to use cuda:0


In [2]:
from src.chatbone import HFPipelineChatModel
chat_model = HFPipelineChatModel(hf_pipeline=hf_pipeline)

In [3]:
from transformers import TextStreamer
streamer = TextStreamer(hf_pipeline.tokenizer,
                        skip_prompt=True,
                        skip_special_tokens=True)
chat_model.invoke("tell me a joke",
                  pipeline_kwargs = dict(streamer =streamer, return_full_text=True), debug = True
                  )

Sure! Here's a light-hearted joke for you:

Why don't scientists trust atoms?

Because they make up everything!
------------START DEBUG----------
messages
[HumanMessage(content='tell me a joke', additional_kwargs={}, response_metadata={})]

messages_dict
[{'role': 'user', 'content': 'tell me a joke'}]

llm_input
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
tell me a joke<|im_end|>
<|im_start|>assistant


llm_output
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
tell me a joke<|im_end|>
<|im_start|>assistant
Sure! Here's a light-hearted joke for you:

Why don't scientists trust atoms?

Because they make up everything!

chat_result
generations=[ChatGeneration(text="<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\ntell me a joke<|im_end|>\n<|im_start|>assistant\nSure! Here's a light-he

AIMessage(content="<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\ntell me a joke<|im_end|>\n<|im_start|>assistant\nSure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={}, id='run-b128884c-9992-4abb-bafd-60f5d3d6905b-0')

In [4]:
messages = [
    dict(role = 'users', content = 'what is 3+5+5?'),
    dict(role = 'assistant', content = 'The sum of 3 and 5 and 5 is 13'),
    dict(role='users', content = "So how about 3+8")
]
chat_model.invoke(messages,
                  pipeline_kwargs = dict(streamer =streamer, return_full_text=True),debug = True
                  )


The sum of 3 and 8 is 11.
------------START DEBUG----------
messages
[HumanMessage(content='what is 3+5+5?', additional_kwargs={}, response_metadata={}), AIMessage(content='The sum of 3 and 5 and 5 is 13', additional_kwargs={}, response_metadata={}), HumanMessage(content='So how about 3+8', additional_kwargs={}, response_metadata={})]

messages_dict
[{'role': 'user', 'content': 'what is 3+5+5?'}, {'role': 'assistant', 'content': 'The sum of 3 and 5 and 5 is 13'}, {'role': 'user', 'content': 'So how about 3+8'}]

llm_input
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
what is 3+5+5?<|im_end|>
<|im_start|>assistant
The sum of 3 and 5 and 5 is 13<|im_end|>
<|im_start|>user
So how about 3+8<|im_end|>
<|im_start|>assistant


llm_output
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
what is 3+5+5?<|im_end|>
<|im_start|>assistant
The sum of 3 and 5 and 5 is 

AIMessage(content='<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nwhat is 3+5+5?<|im_end|>\n<|im_start|>assistant\nThe sum of 3 and 5 and 5 is 13<|im_end|>\n<|im_start|>user\nSo how about 3+8<|im_end|>\n<|im_start|>assistant\nThe sum of 3 and 8 is 11.', additional_kwargs={}, response_metadata={}, id='run-609fb25a-a5c1-415d-bbad-da96e3584062-0')

In [5]:
# Tools
def add(a: int, b: int) -> int:
    """
    A function that adds two numbers.
    Args:
        a: The first number to add
        b: The second number to add
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """
     A function that multiplies two numbers.
     Args:
         a: The first number to multiply
         b: The second number to multiply
     """
    return a * b


from pydantic import BaseModel, Field


class JokeFormat(BaseModel):
    """
    The template when a joke is generated. All the joke has to match this format.
    Args:
        setup: Question to set up the joke.
        punchline: The twist answer to make surprising.
    """
    setup: str = Field(description='Question to setup the joke')
    punchline: str = Field(description='The twist answer to make surprising.')


def joke_format(setup: str, punchline: str):
    """
    The template when a joke is generated. All the joke has to match this format.
    Args:
        setup: Question to set up the joke.
        punchline: The twist answer to make surprising.
    """
    return JokeFormat(setup=setup, punchline=punchline)


tools = [add, multiply]

In [6]:
chat_with_tools = chat_model.bind_tools(tools)
chat_with_tools.kwargs

{'tools': [<function __main__.add(a: int, b: int) -> int>,
  <function __main__.multiply(a: int, b: int) -> int>]}

In [7]:
from langchain_core.messages import HumanMessage
his_messages = [
    # dict(role = 'users', content = 'what is 3+5+5?'),
    # dict(role = 'assistant', content = 'The sum of 3 and 5 and 5 is 13'),
    HumanMessage("what is 3+8 and 3*8 equal to ?")
]
tool_output = chat_with_tools.invoke(his_messages,
                       pipeline_kwargs = dict(streamer =streamer),
                       # debug =True
                       )
his_messages.append(tool_output)
his_messages

<tool_call>
{"name": "add", "arguments": {"a": 3, "b": 8}}
</tool_call>
<tool_call>
{"name": "multiply", "arguments": {"a": 3, "b": 8}}
</tool_call>


[HumanMessage(content='what is 3+8 and 3*8 equal to ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-4d7ec1c3-abb3-4932-be61-e3ab2bd5f565-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 8}, 'id': '2025-02-09 22:38:32', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': '2025-02-09 22:38:32', 'type': 'tool_call'}])]

In [8]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState


node = ToolNode([multiply,add])

m = MessagesState(messages = his_messages)
a = node.invoke(m)
print(a)
his_messages.extend(a['messages'])
his_messages

{'messages': [ToolMessage(content='11', name='add', tool_call_id='2025-02-09 22:38:32'), ToolMessage(content='24', name='multiply', tool_call_id='2025-02-09 22:38:32')]}


[HumanMessage(content='what is 3+8 and 3*8 equal to ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-4d7ec1c3-abb3-4932-be61-e3ab2bd5f565-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 8}, 'id': '2025-02-09 22:38:32', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': '2025-02-09 22:38:32', 'type': 'tool_call'}]),
 ToolMessage(content='11', name='add', tool_call_id='2025-02-09 22:38:32'),
 ToolMessage(content='24', name='multiply', tool_call_id='2025-02-09 22:38:32')]

In [9]:
ai_mess = chat_with_tools.invoke(his_messages,
                       pipeline_kwargs = dict(streamer =streamer, return_full_text=True),
                       debug =True
                       )

3 + 8 equals 11, and 3 * 8 equals 24.
------------START DEBUG----------
messages
[HumanMessage(content='what is 3+8 and 3*8 equal to ?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-4d7ec1c3-abb3-4932-be61-e3ab2bd5f565-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 8}, 'id': '2025-02-09 22:38:32', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': '2025-02-09 22:38:32', 'type': 'tool_call'}]), ToolMessage(content='11', name='add', tool_call_id='2025-02-09 22:38:32'), ToolMessage(content='24', name='multiply', tool_call_id='2025-02-09 22:38:32')]

messages_dict
[{'role': 'user', 'content': 'what is 3+8 and 3*8 equal to ?'}, {'role': 'assistant', 'content': '', 'tool_calls': [{'type': 'function', 'id': '2025-02-09 22:38:32', 'function': {'name': 'add', 'arguments': {'a': 3, 'b': 8}}}, {'type': 'function', 'id': '2025-02-09 22:38:32', 'function': {'name': 'multiply', 'arguments': {'a':

In [10]:
print(ai_mess.content)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "add", "description": "A function that adds two numbers.", "parameters": {"type": "object", "properties": {"a": {"type": "integer", "description": "The first number to add"}, "b": {"type": "integer", "description": "The second number to add"}}, "required": ["a", "b"]}, "return": {"type": "integer"}}}
{"type": "function", "function": {"name": "multiply", "description": "A function that multiplies two numbers.", "parameters": {"type": "object", "properties": {"a": {"type": "integer", "description": "The first number to multiply"}, "b": {"type": "integer", "description": "The second number to multiply"}}, "required": ["a", "b"]}, "return": {"type": "integer"}}}
</tools>

For each function call, return a 